# Transcriptomic analysis reveals the roles of microtubule-related genes and transcription factors in fruit trichome regulation in cucumber (Cucumis sativus L.)

Pipeline: Firstly, we carried out RNA-seq gene differential expression analysis on cucumber two different trichome datasets. Secondly, we carried out funcitonal analysis of differentially expressed genes to identify their functions and involved pathways.

## Part one. Genes differential expression analysis between two different cucumber trichome varieties.

<img src="https://github.com/fengkuangbaozha/maize_nitrogen/blob/fig/maize_fig1.png?raw=true" style="height: 300px;" />
Figure 1. Pipeline of differentially expressed genes identification.

### 1. We cleaned the reads with SolexaQA and Cutadapter, mapped reads to the cucumber genome (ChineseLong v2) with tophat, calculated reads counts with htseq-count.

```bash
qsub -q small.q -t 1-2 /psc/home/ydsun/zhaojy_trichome/script/assemble_single.sh /psc/home/ydsun/genome/cucumis_sativus/csa_ChineseLong.v2.gtf /psc/home/ydsun/genome/cucumis_sativus/csa_ChineseLong.v2 /psc/home/ydsun/zhaojy_trichome/data/ 1983-G
qsub -q medium.q -t 1-2 /psc/home/ydsun/zhaojy_trichome/script/assemble_single.sh /psc/home/ydsun/genome/cucumis_sativus/csa_ChineseLong.v2.gtf /psc/home/ydsun/genome/cucumis_sativus/csa_ChineseLong.v2 /psc/home/ydsun/zhaojy_trichome/data/ 1983-H
```

###### Reads statistics.

```sh
bash /psc/home/ydsun/zhaojy_trichome/script/reads_statistics.sh  /psc/home/ydsun/zhaojy_trichome/data
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig1.png?raw=true" style="width: 600px;" />
Table 1. Statisctics of RNA-seq samples.

### 2. We test replicates quality with PCA and Pairwise analysis.

```sh
cd /psc/bioinformatics/sunyd/zhaojy_trichome/process
Rscript /psc/bioinformatics/sunyd/zhaojy_trichome/script/pca_pairwise_3rep.R
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig1_3.png?raw=true" style="height: 500px;" />
<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig1_2.png?raw=true" style="height: 500px;" />
Figure 2. PCA and Pairwise analysis of RNA-seq samples.

### 3. We performed differential expression (DE) analysis with edgeR between two different cucumber trichome varieties and compared with DGE data analysis published in JXB.

```sh
bash /psc/home/ydsun/zhaojy_trichome/script/replicate_all_de.sh /psc/home/ydsun/zhaojy_trichome/process
bash /psc/home/ydsun/zhaojy_trichome/script/de_updown.sh /psc/home/ydsun/zhaojy_trichome/process  ######Up and down DE genes and expression ###
Rscript ../../script/venn_snp.R rnaseq.de.txt develop_stage.de.txt ck_tbh.de.txt venn_RNAseq_DGE 0.06 0.06 0.04
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig2.png?raw=true" style="height: 300px;" />
Figure 3. Overlap of differentially expressed genes with DGE data analysis published in JXB.

## Part Two. Analysis of transcription factors.

### 1. We identified DE genes involved pathways with Mapman.

```sh
cd /psc/home/ydsun/zhaojy_trichome/process
for i in *diff.txt; do j=`echo $i |sed 's/diff/diff.mapman/g'`; cut -f1,2 $i |sed '1d' > $j; done    ####only keep name and fold change
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig3.png?raw=true" style="height: 500px;" />
Figure 4. Differentially expressed genes are enriched in transcription factors.

### 2. Classification of transcription factors.

```sh
grep -Fwf <(awk '{if($3<$5) print $0}' topgo/categorydetail_all.trichome.RNA.txt |cut -d "(" -f1) ~/genome/cucumis_sativus/csa_ChineseLong.v2.mapman.mercator.txt |cut -f2,3 |grep "RNA.regulation of transcription" |sed 's/ /_/g' |cut -f1 |sort |uniq -c |sed 's/RNA.regulation_of_transcription.//g' |awk '{OFS="\t";print $2,$1}' |sort -k2 -g -r > tf_classification
paste <(cut -d_ -f1 tf_classification) tf_classification |sed "s/'//g" > tf_classification1
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig4.png?raw=true" style="width: 600px;" />
Figure 5. Classification of transcription factors.

### 3. GO enrichment analysis of up regulated genes with topgo package.

```sh
bash /psc/home/ydsun/zhaojy_trichome/script/topgo.sh /psc/home/ydsun/zhaojy_trichome/process/topgo /psc/home/ydsun/genome/cucumis_sativus/cuc.map
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig5.png?raw=true" style="width: 800px;" />
Figure 6.  GO enrichment of up regualted genes.

## Part Three. Analysis of Microtube Related Genes

### 1. We identified DE genes involved pathways with Mapman.

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig6.png?raw=true" style="height: 800px;" />
Figure 7.  Differentially expressed genes are enriched in cell wall, cell division and cell cycle.

### 2. Functional category enrichment of differentially expressed genes.

```sh
bash /psc/home/ydsun/zhaojy_trichome/script/category_enrich.sh /psc/home/ydsun/genome/cucumis_sativus/csa_ChineseLong.v2.mapman.category.id /psc/home/ydsun/zhaojy_trichome/process/topgo
bash /psc/home/ydsun/zhaojy_trichome/script/category_detail_specific.sh /psc/home/ydsun/zhaojy_trichome/process/topgo trichome _all 1-6
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig7.png?raw=true" style="width: 800px;" />
Figure 8.  Functional category enrichment of differentially expressed genes.

### 3. GO enrichment analysis of down regulated genes with topgo package.

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig8.png?raw=true" style="width: 500px;" />
Figure 9.  GO enrichment of down regulated genes.

## Part Four. KEGG pathway analysis of differentially expressed genes.

```sh
bash /psc/home/ydsun/zhaojy_trichome/script/kegg.sh /psc/home/ydsun/zhaojy_trichome/process/topgo /psc/home/ydsun/genome/cucumis_sativus/csa_ChineseLong.v2.kegg
```

<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig9.png?raw=true" style="width: 500px;" />
<img src="https://github.com/fengkuangbaozha/cucumber_trichome/blob/fig/fig10.png?raw=true" style="width: 500px;" />
Figure 10.  KEGG pathway analysis of differentially expressed genes.

### Notice: our partner is responsible for the sample preparation, RNA sequencing and following experimental validation.